# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [1]:
# !wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip

In [2]:
# !unzip /content/stanfordSentimentTreebank.zip

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/stanfordSentimentTreebank/datasetSentences.txt', sep="\t", header=0)
labels = pd.read_csv('/content/stanfordSentimentTreebank/sentiment_labels.txt', sep="|", header=0)

labels.rename(columns={"phrase ids": "idx", "sentiment values": "labels"}, inplace=True)
data.rename(columns={"sentence_index": "idx", "sentence": "sentences"}, inplace=True)

df = pd.merge(data, labels, on=['idx'])

df.drop(['idx'], axis=1, inplace=True)
df.head()

bins = list(np.arange(0.0, 1.0, 0.04))
bins.append(1.0)

df['labels_bins'] = pd.cut(df.labels, bins=bins, include_lowest=True, labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])

df.drop(['labels'], axis=1, inplace=True)

df.rename(columns={"labels_bins": "labels"}, inplace=True)

df['sentences'] = df.sentences.map(lambda x: x[::-1])

df.head()

,sentences,labels
0,". lageS nevetS ro emmaD naV dualC-naeJ , regge...",13
1,. htrae-elddiM s' neikloT .R.R.J fo noisiv ded...,12
2,cipoib dipet-oot tub evitceffE,13
3,". trats ot ecalp doog a si ibasaW , nuf evah o...",11
4,. eno ekil leef t'n seod ti taht devresbo ylne...,10


In [4]:
df.shape

(11855, 2)

In [5]:
df.labels.value_counts()

13    2901
14    1265
12     866
15     647
16     621
10     593
11     561
8      439
7      433
19     403
9      389
20     383
18     381
17     351
6      321
21     310
5      286
22     162
4      156
23     104
3      103
2       70
24      65
1       23
25      22
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [6]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [7]:
Sentence = data.Field(sequential=True, tokenize='spacy', batch_first=True, include_lengths=True)
Label = data.LabelField(tokenize='spacy', is_target=True, batch_first=True, sequential=False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [8]:
fields = [('sentences', Sentence),('labels', Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [9]:
example = [data.Example.fromlist([df.sentences[i], df.labels[i]], fields) for i in range(df.shape[0])] 

In [10]:
# Creating dataset
# sentenceDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

sentenceDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [11]:
(train, valid) = sentenceDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [12]:
(len(train), len(valid))

(10077, 1778)

An example from the dataset:

In [13]:
vars(train.examples[10])

{'labels': 11,
 'sentences': ['.',
  'raey',
  'lla',
  'ees',
  'ot',
  'ylekil',
  'er',
  "'",
  'uoy',
  'mlif',
  'a',
  'dab',
  'sa',
  'tuoba',
  ',',
  'sdrow',
  'rehto',
  'nI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [14]:
Sentence.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [15]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18830
Size of label vocab :  25
Top 10 words appreared repeatedly : [('.', 9479), (',', 8475), ('eht', 7175), ('dna', 5253), ('a', 5194), ('fo', 5183), ("'", 3961), ('ot', 3608), ('-', 3226), ('s', 3025)]
Labels :  defaultdict(<function _default_unk_index at 0x7f6474d93d08>, {13: 0, 14: 1, 12: 2, 15: 3, 16: 4, 10: 5, 11: 6, 8: 7, 7: 8, 19: 9, 9: 10, 18: 11, 20: 12, 17: 13, 6: 14, 21: 15, 5: 16, 22: 17, 4: 18, 3: 19, 23: 20, 2: 21, 24: 22, 1: 23, 25: 24})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentences),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [18]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [20]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 25
num_layers = 5
dropout = 0.1

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [21]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18830, 300)
  (encoder): LSTM(300, 100, num_layers=5, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=100, out_features=25, bias=True)
)
The model has 6,135,525 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [22]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)

    print(pridictions)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [23]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentences   
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [24]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentences
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [25]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 3.117 | Train Acc: 21.51%
	 Val. Loss: 3.051 |  Val. Acc: 23.59% 

	Train Loss: 3.040 | Train Acc: 24.64%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.63%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.64%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.64%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.65%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.65%
	 Val. Loss: 3.050 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.66%
	 Val. Loss: 3.051 |  Val. Acc: 23.59% 

	Train Loss: 3.039 | Train Acc: 24.67%
	 Val. Loss: 3.052 |  Val. Acc: 23.59% 

	Train Loss: 3.038 | Train Acc: 24.70%
	 Val. Loss: 3.053 |  Val. Acc: 23.53% 

	Train Loss: 3.038 | Train Acc: 24.84%
	 Val. Loss: 3.052 |  Val. Acc: 23.47% 

	Train Loss: 3.037 | Train Acc: 24.95%
	 Val. Loss: 3.053 |  Val. Acc: 23.59% 

	Train Loss: 3.036 | Train Acc: 25.01%
	

## Model Testing

In [26]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [27]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [28]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [29]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [30]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [33]:
!pip install google_trans_new

In [55]:
import random
import google_trans_new
from google_trans_new import google_translator

translator = google_translator()
sentence = ['The drank dog slept on my coffee table legs']

available_langs = list(google_trans_new.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, lang_tgt=trans_lang) 
print(translations)

translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
print(translations_en_random)

Translating to hebrew
['הכלב השתוי ישן על רגלי שולחן הקפה שלי'] 
['The drunken dog is sleeping on my coffee table'] 
